<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/NLP/%EB%AC%B8%EC%9E%A5%EC%9D%BC%EA%B4%80%EC%84%B1(%EC%9D%91%EC%A7%91%EC%84%B1)_%ED%8C%90%EB%8B%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentence transfomers 구성 및 훈련

In [33]:
!pip install sentence_transformers datasets

In [34]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [35]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [36]:
model_name = "monologg/koelectra-base-v3-discriminator"
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [37]:
embedding_model = models.Transformer(model_name)

2021-10-27 05:09:37 - Attempting to acquire lock 140034389802576 on /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c.lock
2021-10-27 05:09:37 - Lock 140034389802576 acquired on /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c.lock


Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

2021-10-27 05:09:37 - Attempting to release lock 140034389802576 on /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c.lock
2021-10-27 05:09:37 - Lock 140034389802576 released on /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c.lock
2021-10-27 05:09:37 - Attempting to acquire lock 140034389683344 on /root/.cache/huggingface/transformers/efc891feb2c6ce810e4350a7fb9d7f9f48d16b78cc24f95c34bd2d5d1d4bb9c4.20d7b162691634a30f634433042d1bfde6dea4760555a95e166bfa3a3b415682.lock
2021-10-27 05:09:37 - Lock 140034389683344 acquired on /root/.cache/huggingface/transformers/efc891feb2c6ce810e4350a7fb9d7f9f48d16b78cc24f95c34bd2d5d1d4bb9c4.20d7b162691634a30f634433042d1bfde6dea4760555a95e166bfa3a3b415682.lock


Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

2021-10-27 05:09:53 - Attempting to release lock 140034389683344 on /root/.cache/huggingface/transformers/efc891feb2c6ce810e4350a7fb9d7f9f48d16b78cc24f95c34bd2d5d1d4bb9c4.20d7b162691634a30f634433042d1bfde6dea4760555a95e166bfa3a3b415682.lock
2021-10-27 05:09:53 - Lock 140034389683344 released on /root/.cache/huggingface/transformers/efc891feb2c6ce810e4350a7fb9d7f9f48d16b78cc24f95c34bd2d5d1d4bb9c4.20d7b162691634a30f634433042d1bfde6dea4760555a95e166bfa3a3b415682.lock


...

2021-10-27 05:09:57 - Attempting to acquire lock 140034389171280 on /root/.cache/huggingface/transformers/f2c12b0a8c12c754523b6ed45c1681b4495050720ade30bfe2345e5573d2e141.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock
2021-10-27 05:09:57 - Lock 140034389171280 acquired on /root/.cache/huggingface/transformers/f2c12b0a8c12c754523b6ed45c1681b4495050720ade30bfe2345e5573d2e141.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock


Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

2021-10-27 05:09:57 - Attempting to release lock 140034389171280 on /root/.cache/huggingface/transformers/f2c12b0a8c12c754523b6ed45c1681b4495050720ade30bfe2345e5573d2e141.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock
2021-10-27 05:09:57 - Lock 140034389171280 released on /root/.cache/huggingface/transformers/f2c12b0a8c12c754523b6ed45c1681b4495050720ade30bfe2345e5573d2e141.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock
2021-10-27 05:09:58 - Attempting to acquire lock 140034389172176 on /root/.cache/huggingface/transformers/896eae0fbbb6c2981e5d4559f5d7fc0deb03e81c292abbe5c89b5bb1cf1294b3.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock
2021-10-27 05:09:58 - Lock 140034389172176 acquired on /root/.cache/huggingface/transformers/896eae0fbbb6c2981e5d4559f5d7fc0deb03e81c292abbe5c89b5bb1cf1294b3.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock


Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

2021-10-27 05:09:58 - Attempting to release lock 140034389172176 on /root/.cache/huggingface/transformers/896eae0fbbb6c2981e5d4559f5d7fc0deb03e81c292abbe5c89b5bb1cf1294b3.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock
2021-10-27 05:09:58 - Lock 140034389172176 released on /root/.cache/huggingface/transformers/896eae0fbbb6c2981e5d4559f5d7fc0deb03e81c292abbe5c89b5bb1cf1294b3.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock


In [38]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [39]:
model = SentenceTransformer(modules=[embedding_model, pooler])

2021-10-27 05:09:59 - Use pytorch device: cuda


In [40]:
datasets = load_dataset("klue", "sts")

2021-10-27 05:10:00 - Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [42]:
testsets = load_dataset("kor_nlu", "sts")

2021-10-27 05:10:01 - Reusing dataset kor_nlu (/root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [44]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [45]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [46]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2021-10-27 05:10:03 - Warmup-steps: 146


In [47]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-27 05:16:15 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-10-27 05:16:19 - Cosine-Similarity :	Pearson: 0.5885	Spearman: 0.5800
2021-10-27 05:16:19 - Manhattan-Distance:	Pearson: 0.5858	Spearman: 0.5679
2021-10-27 05:16:19 - Euclidean-Distance:	Pearson: 0.5883	Spearman: 0.5691
2021-10-27 05:16:19 - Dot-Product-Similarity:	Pearson: 0.4946	Spearman: 0.4800
2021-10-27 05:16:19 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-27_05-09-37


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-27 05:22:28 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2021-10-27 05:22:32 - Cosine-Similarity :	Pearson: 0.6893	Spearman: 0.6856
2021-10-27 05:22:32 - Manhattan-Distance:	Pearson: 0.6706	Spearman: 0.6550
2021-10-27 05:22:32 - Euclidean-Distance:	Pearson: 0.6715	Spearman: 0.6563
2021-10-27 05:22:32 - Dot-Product-Similarity:	Pearson: 0.6144	Spearman: 0.6017
2021-10-27 05:22:32 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-27_05-09-37


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-27 05:28:44 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2021-10-27 05:28:48 - Cosine-Similarity :	Pearson: 0.7012	Spearman: 0.6997
2021-10-27 05:28:48 - Manhattan-Distance:	Pearson: 0.6894	Spearman: 0.6747
2021-10-27 05:28:48 - Euclidean-Distance:	Pearson: 0.6912	Spearman: 0.6763
2021-10-27 05:28:48 - Dot-Product-Similarity:	Pearson: 0.6315	Spearman: 0.6160
2021-10-27 05:28:48 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-27_05-09-37


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-27 05:34:58 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2021-10-27 05:35:03 - Cosine-Similarity :	Pearson: 0.7079	Spearman: 0.7073
2021-10-27 05:35:03 - Manhattan-Distance:	Pearson: 0.6916	Spearman: 0.6777
2021-10-27 05:35:03 - Euclidean-Distance:	Pearson: 0.6935	Spearman: 0.6796
2021-10-27 05:35:03 - Dot-Product-Similarity:	Pearson: 0.6399	Spearman: 0.6266
2021-10-27 05:35:03 - Save model to output/training_klue_sts_monologg-koelectra-base-v3-discriminator-2021-10-27_05-09-37


# 형태소 분석 및 품사 필터링 (명사, 동사, 형용사)

In [48]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-10-27 05:35:09--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::34cc:ea4a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=d1mxMZzN8gxNILpTjPTOuDY89V4%3D&Expires=1635314

In [49]:
from konlpy.tag import *
okt = Okt()
mecab = Mecab()

In [50]:
print(mecab.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print(mecab.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print(mecab.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))

['코딩', '당신', '연휴', '여행']
['열심히', '코딩', '한', '당신', ',', '연휴', '에', '는', '여행', '을', '가', '봐요']
[('열심히', 'MAG'), ('코딩', 'NNG'), ('한', 'XSA+ETM'), ('당신', 'NP'), (',', 'SC'), ('연휴', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가', 'VV'), ('봐요', 'EC+VX+EC')]


In [51]:
document = ''
clean_words = []
for word in okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요", stem=True):
  if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
    clean_words.append(word[0])
print(clean_words)
document = ' '.join(clean_words)

['코딩', '당신', '연휴', '여행', '가보다']


In [52]:
docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]
document_embeddings = model.encode(docs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
result_arr = []
result = []
for sentence in docs:
  clean_words = []
  for word in okt.pos(sentence, stem=True):
    if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
      clean_words.append(word[0])
  print(clean_words)
  temp_sentence = ' '.join(clean_words)
  result_arr.append(clean_words)
  result.append(temp_sentence)

['손흥민', '강원도', '춘천시', '후평동', '아버지', '손웅정', '어머니', '길다', '자의', '차남', '태어나다', '곳', '자르다']
['형', '손흥윤']
['춘천', '부안', '초등학교', '졸업', '하다', '춘천', '후평', '중학교', '입학', '후', '학년', '때', '원주', '민', '관중', '학교', '축구', '부', '들어가다', '위해', '전학', '하다', '졸업', '하다', '당시', '서울', '팀', '이다', '동북', '고등학교', '축구', '부', '선수', '활동', '중', '축구', '협회', '우수', '선수', '해외', '유학', '프로젝트', '선발', '되어다', '독일', '분데스리가', '함부르크', '유', '소년', '팀', '입단', '하다']
['함부르크', '유', '스팀', '주전', '공격수', '네덜란드', '열리다', '개국', '경기', '게임', '출전', '골', '터뜨리다']
['유학', '후', '한국', '돌아오다', '후', '개막', '월드컵', '출전', '하다', '골', '터', '트리', '한국', '강', '이끌다']
['함부르크', '정식', '유', '소년', '팀', '선수', '계약', '체결', '하다', '독일', '리그', '경기', '골', '넣다', '군', '리그', '출전', '시작', '하다']
['독일', '리그', '손흥민', '경기', '골', '부', '리그', '경기', '골', '넣다', '재능', '인정받다', '세', '나이', '함부르크', '군', '팀', '훈련', '참가', '프리', '시즌', '활약', '함부르크', '정식', '계약', '하다', '후', '세', '함부르크', '군', '소속', '독일', '분데스리가', '데뷔', '하다']


In [54]:
result_arr

[['손흥민',
  '강원도',
  '춘천시',
  '후평동',
  '아버지',
  '손웅정',
  '어머니',
  '길다',
  '자의',
  '차남',
  '태어나다',
  '곳',
  '자르다'],
 ['형', '손흥윤'],
 ['춘천',
  '부안',
  '초등학교',
  '졸업',
  '하다',
  '춘천',
  '후평',
  '중학교',
  '입학',
  '후',
  '학년',
  '때',
  '원주',
  '민',
  '관중',
  '학교',
  '축구',
  '부',
  '들어가다',
  '위해',
  '전학',
  '하다',
  '졸업',
  '하다',
  '당시',
  '서울',
  '팀',
  '이다',
  '동북',
  '고등학교',
  '축구',
  '부',
  '선수',
  '활동',
  '중',
  '축구',
  '협회',
  '우수',
  '선수',
  '해외',
  '유학',
  '프로젝트',
  '선발',
  '되어다',
  '독일',
  '분데스리가',
  '함부르크',
  '유',
  '소년',
  '팀',
  '입단',
  '하다'],
 ['함부르크',
  '유',
  '스팀',
  '주전',
  '공격수',
  '네덜란드',
  '열리다',
  '개국',
  '경기',
  '게임',
  '출전',
  '골',
  '터뜨리다'],
 ['유학',
  '후',
  '한국',
  '돌아오다',
  '후',
  '개막',
  '월드컵',
  '출전',
  '하다',
  '골',
  '터',
  '트리',
  '한국',
  '강',
  '이끌다'],
 ['함부르크',
  '정식',
  '유',
  '소년',
  '팀',
  '선수',
  '계약',
  '체결',
  '하다',
  '독일',
  '리그',
  '경기',
  '골',
  '넣다',
  '군',
  '리그',
  '출전',
  '시작',
  '하다'],
 ['독일',
  '리그',
  '손흥민',
  '경기',
  '골',
  '부',
  '리그',
  '경기',
  '

# 코사인 유사도 기반으로 한 국소적 응집도 측정

In [55]:
document_embeddings = model.encode(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
query = "손흥민은 어린 나이에 유럽에 진출하였다."
query_embedding = model.encode(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

top_k = 5

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

입력 문장: 손흥민은 어린 나이에 유럽에 진출하였다.

<입력 문장과 유사한 5 개의 문장>

1: 독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다. (유사도: 0.4049)

2: 그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다. (유사도: 0.3798)

3: 함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다. (유사도: 0.3779)

4: 1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다. (유사도: 0.3673)

5: 춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다. (유사도: 0.2330)



In [58]:
query_a = "춘천 부안 초등학교 졸업 하다 춘천 후평 중학교 입학 후 학년 때 원주 민 관중 학교 축구 부 들어가다 위해 전학 하다 졸업 하다 당시 서울 팀 이다 동북 고등학교 축구 부 선수 활동 중 축구 협회 우수 선수 해외 유학 프로젝트 선발 되어다 독일 분데스리가 함부르크 유 소년 팀 입단 하다"
query_embedding_a = model.encode(query_a)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [59]:
query_b = "형 손흥윤"
query_embedding_b = model.encode(query_b)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
import numpy
all_coherence = []
for i, sentence in enumerate(result):
  print(i)
  if(i==len(result)-1): 
    break
  query_embedding_a = model.encode(result[i])
  query_embedding_b = model.encode(result[i+1])

  # 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
  cos_scores = util.pytorch_cos_sim(query_embedding_a, query_embedding_b)[0]
  print(result[i], '와' ,result[i+1], ' 의 코사인 유사도 :', cos_scores)
  all_coherence.append(cos_scores)
print('평균 코사인 유사도: ',numpy.mean(all_coherence))
print('표준 편차: ',numpy.std(all_coherence))

0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

손흥민 강원도 춘천시 후평동 아버지 손웅정 어머니 길다 자의 차남 태어나다 곳 자르다 와 형 손흥윤  의 코사인 유사도 : tensor([0.3003])
1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

유학 후 한국 돌아오다 후 개막 월드컵 출전 하다 골 터 트리 한국 강 이끌다 와 함부르크 정식 유 소년 팀 선수 계약 체결 하다 독일 리그 경기 골 넣다 군 리그 출전 시작 하다  의 코사인 유사도 : tensor([0.7648])
5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

함부르크 정식 유 소년 팀 선수 계약 체결 하다 독일 리그 경기 골 넣다 군 리그 출전 시작 하다 와 독일 리그 손흥민 경기 골 부 리그 경기 골 넣다 재능 인정받다 세 나이 함부르크 군 팀 훈련 참가 프리 시즌 활약 함부르크 정식 계약 하다 후 세 함부르크 군 소속 독일 분데스리가 데뷔 하다  의 코사인 유사도 : tensor([0.8394])
6
평균 코사인 유사도:  0.5110176
표준 편차:  0.26133057


# 상호 참조 관계 (단어 포함 유사도) 기반으로 한 국소적 응집도 측정

In [61]:
import numpy
all_coherence = []
for i, sentence in enumerate(result_arr):
  print(i)
  if(i==len(result)-1): 
    break
  same_word = [x for x in result_arr[i] if x in result_arr[i+1]]
  # 단어 포함 여부( 상호 참조 관계) 확인 후 평가
  print(result[i], '와' ,result[i+1], ' 의 상호 참조 관계 :', min(len(same_word),1))
  all_coherence.append(min(len(same_word),1))
print('국소 응집성: ',numpy.mean(all_coherence))
print('표준 편차: ',numpy.std(all_coherence))

0
손흥민 강원도 춘천시 후평동 아버지 손웅정 어머니 길다 자의 차남 태어나다 곳 자르다 와 형 손흥윤  의 상호 참조 관계 : 0
1
형 손흥윤 와 춘천 부안 초등학교 졸업 하다 춘천 후평 중학교 입학 후 학년 때 원주 민 관중 학교 축구 부 들어가다 위해 전학 하다 졸업 하다 당시 서울 팀 이다 동북 고등학교 축구 부 선수 활동 중 축구 협회 우수 선수 해외 유학 프로젝트 선발 되어다 독일 분데스리가 함부르크 유 소년 팀 입단 하다  의 상호 참조 관계 : 0
2
춘천 부안 초등학교 졸업 하다 춘천 후평 중학교 입학 후 학년 때 원주 민 관중 학교 축구 부 들어가다 위해 전학 하다 졸업 하다 당시 서울 팀 이다 동북 고등학교 축구 부 선수 활동 중 축구 협회 우수 선수 해외 유학 프로젝트 선발 되어다 독일 분데스리가 함부르크 유 소년 팀 입단 하다 와 함부르크 유 스팀 주전 공격수 네덜란드 열리다 개국 경기 게임 출전 골 터뜨리다  의 상호 참조 관계 : 1
3
함부르크 유 스팀 주전 공격수 네덜란드 열리다 개국 경기 게임 출전 골 터뜨리다 와 유학 후 한국 돌아오다 후 개막 월드컵 출전 하다 골 터 트리 한국 강 이끌다  의 상호 참조 관계 : 1
4
유학 후 한국 돌아오다 후 개막 월드컵 출전 하다 골 터 트리 한국 강 이끌다 와 함부르크 정식 유 소년 팀 선수 계약 체결 하다 독일 리그 경기 골 넣다 군 리그 출전 시작 하다  의 상호 참조 관계 : 1
5
함부르크 정식 유 소년 팀 선수 계약 체결 하다 독일 리그 경기 골 넣다 군 리그 출전 시작 하다 와 독일 리그 손흥민 경기 골 부 리그 경기 골 넣다 재능 인정받다 세 나이 함부르크 군 팀 훈련 참가 프리 시즌 활약 함부르크 정식 계약 하다 후 세 함부르크 군 소속 독일 분데스리가 데뷔 하다  의 상호 참조 관계 : 1
6
국소 응집성:  0.6666666666666666
표준 편차:  0.4714045207910317


In [62]:
all_coherence

[0, 0, 1, 1, 1, 1]